In [19]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

In [20]:
# if v1 == v2 (number), target = 0 else target = 1 
def compare_vertex(qc, qr, v1, v2, target):
    qc.x(qr[2*v2])
    qc.x(qr[2*v2+1])
    qc.ccx(qr[2*v1], qr[2*v1+1], target)
    qc.ccx(qr[2*v1+1], qr[2*v2], target)
    qc.ccx(qr[2*v2], qr[2*v2+1], target)
    qc.ccx(qr[2*v1], qr[2*v2+1], target)
    qc.x(qr[2*v2+1])
    qc.x(qr[2*v2])
    #
    qc.x(qr[target])

In [21]:
def compare_vertex_inv(qc, qr, v1, v2, target):
    qc.x(qr[target])
    #
    qc.x(qr[2*v2])
    qc.x(qr[2*v2+1])
    qc.ccx(qr[2*v1], qr[2*v2+1], target)
    qc.ccx(qr[2*v2], qr[2*v2+1], target)
    qc.ccx(qr[2*v1+1], qr[2*v2], target)
    qc.ccx(qr[2*v1], qr[2*v1+1], target)
    qc.x(qr[2*v2+1])
    qc.x(qr[2*v2])

In [5]:
# とりあえず動くはず
def simple_oracle(qc, qr):
    # vs = [0, 1, 2, 3, 4, 5, 6, 7]
    lst = [[0, 1], [2, 3], [4, 5], [6, 7], [2, 6], [0, 4], [1, 5], [3, 7], [0, 2], [1, 3], [4, 6], [5, 7]]
    target = 16
    for elm in lst:
        assert(len(elm)==2)
        compare_vertex(qc, qr, elm[0], elm[1], target)
        target+=1
    assert(target<=29)
    dst = 29
    qc.mct(qr[16:target], qr[dst], [], mode='noancilla')
    for elm in (reversed(lst)):
        target-=1
        compare_vertex_inv(qc, qr, elm[0], elm[1], target)

In [ ]:
# ここから先書きかけ

In [1]:
# vs の全点対について compare して target に AND 格納
def compare_vertexes(qc, qr, vs, target):
    sz = len(vs)
    index = 16
    for i in range(sz):
        for j in range(i+1,sz,1):
            compare_vertex(qc, qr, vs[i], vs[j], index)
            index+=1
    assert(index<=target)
    qc.mct(qr[16:index], qr[target], [], mode='noancilla')
    for i in range(sz-1, -1, -1):
        for j in range(sz-1, i, -1):
            index-=1
            compare_vertex_inv(qc, qr, vs[i], vs[j], index)

In [2]:
def compare_vertexes_inv(qc, qr, vs, target):
    compare_vertexes(qc, qr, vs, target)

In [24]:
def oracle_list(qc, qr, lst, target):
    index=22
    for elm in lst:
        compare_vertexes(qc, qr, elm, index)
        index+=1
    qc.mct(qr[22:index], qr[target], [], mode='noancilla')
    for elm in (reversed(lst)):
        index-=1
        compare_vertexes(qc, qr, elm, index)

In [25]:
def oracle_list_inv(qc, qr, list, target):
    # TODO
    print("TODO")    

In [26]:
def oracle(qc, qr):
    index = 22
    oracle_list(qc, qr, [[0,1], [2,3], [4,5], [6,7]], 26)
    oracle_list(qc, qr, [[0,4], [1,5], [2,6], [3,7]], 27)
    oracle_list(qc, qr, [[0,2], [1,3], [4,6], [5,7]], 28)
    qc.mct(qr[26:29], qr[29], [], mode='noancilla')
    oracle_list_inv(qc, qr, [[0,2], [1,3], [4,6], [5,7]], 28)
    oracle_list_inv(qc, qr, [[0,4], [1,5], [2,6], [3,7]], 27)
    oracle_list_inv(qc, qr, [[0,1], [2,3], [4,5], [6,7]], 26)